In [52]:
import operator

# input: a set of data with match results
# output: a table with final standings

def extract_players(my_matches):
    players = []
    for i in my_matches:
        players.append(i[0]["player"])
        players.append(i[1]["player"])
    return list(set(players))

def normalized_score(player_score):
    deals = player_score["won"] + player_score["lost"]
    win_rate =(player_score["won"]+0.0)/deals
    return player_score["points"]*100 + round(win_rate *100, 0)

def get_sorted_list(players, player_points):
    standings = {}
    for i in players:
        score = normalized_score(player_points[i])
        standings[i] = score
    sorted_standings = sorted(standings.items(), key=operator.itemgetter(1), reverse=True)

    return sorted_standings
        
    

match_data = [
     [{"player": "CAS", "score": 2}, {"player": "BAR", "score": 1}],
     [{"player": "CAS", "score": 2}, {"player": "KUB", "score": 1}],
     [{"player": "CAS", "score": 2}, {"player": "MAT", "score": 1}],
     [{"player": "BAR", "score": 2}, {"player": "KUB", "score": 1}],
     [{"player": "BAR", "score": 2}, {"player": "MAT", "score": 1}],
     [{"player": "MAT", "score": 2}, {"player": "KUB", "score": 1}] 
]

for i in match_data:
    if i[0]["player"] == "CAS":
        print i[0]["score"]
        
players = extract_players(match_data)

player_points = {}
for i in players:
    player_points[i] = {"points": 0, "won": 0, "lost": 0}
    
for i in players:
    for j in match_data:
        if j[0]["player"] == i:
            player_points[i]["won"] +=j[0]["score"]
            if j[0]["score"] == 2:
                player_points[i]["points"] += 3
            player_points[i]["lost"] +=j[1]["score"]
        if j[1]["player"] == i:
            player_points[i]["won"] +=j[1]["score"]
            if j[1]["score"] == 2:
                player_points[i]["points"] += 3
            player_points[i]["lost"] +=j[0]["score"]
                
for i in players:
    print player_points[i]
    
standings = get_sorted_list(players, player_points)

for i in standings:
    print i[]



2
2
2
{'points': 9, 'won': 6, 'lost': 3}
{'points': 6, 'won': 5, 'lost': 4}
{'points': 0, 'won': 3, 'lost': 6}
{'points': 3, 'won': 4, 'lost': 5}
('CAS', 967.0)
('BAR', 656.0)
('MAT', 344.0)
('KUB', 33.0)
